In [3]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import glob as glob
from torch import topk
import pandas as pd
from collections.abc import Iterable
from torch.utils.data import Dataset, DataLoader
import itertools
import tqdm
import copy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import seaborn as sns
# imports from captum library
from captum.attr import LayerConductance, LayerActivation, LayerIntegratedGradients
from captum.attr import IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, FeatureAblation

In [4]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [7]:
df = pd.read_csv('data/depths_added.csv')



In [8]:
df

,l_hand_min_x,l_hand_min_y,l_hand_width,l_hand_height,r_hand_min_x,r_hand_min_y,r_hand_width,r_hand_height,object_min_x,object_min_y,...,r19x,r19y,r20x,r20y,r21x,r21y,picture_name,grasp,lh_depth_dist,rh_depth_dist
0,1920,0,0,0,1001,682,142,64,869,576,...,100,-3,86,-14,78,-27,salad16085.png,none,-128.411926,-2014.397278
1,1920,1080,0,0,1539,608,209,259,667,243,...,504,-358,479,-333,451,-321,salad09942.png,none,-10972.939575,-6049.359009
2,1171,587,118,168,1920,1080,0,0,712,351,...,-334,422,-334,422,-334,422,salad11706.png,none,-3838.286377,-18578.716064
3,1326,521,101,62,0,1080,0,0,1046,267,...,35,403,35,403,35,403,salad04018.png,none,-814.518860,-905.653381
4,1118,351,182,198,1366,647,105,195,564,484,...,296,-71,273,-59,251,-55,salad16039.png,none,-1761.761719,-1246.901855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4753,972,270,148,176,0,0,0,0,920,926,...,1023,985,1023,985,1023,985,simulation02823.png,precision,877.946289,6519.351683
4754,877,749,188,121,1003,521,135,97,981,603,...,169,116,166,104,166,94,simulation00768.png,precision,-2146.814453,-174.041504
4755,866,400,172,127,0,0,0,0,943,664,...,1004,727,1004,727,1004,727,simulation02895.png,precision,-1189.540039,4339.987671
4756,1174,875,123,53,0,0,0,0,1097,964,...,1222,1022,1222,1022,1222,1022,simulation00537.png,precision,901.244385,6482.765343


In [11]:
cols = list(df.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('grasp')) 
cols.pop(cols.index('picture_name')) 
df = df[cols+['picture_name', 'grasp']] #Create new dataframe with columns in the order you want

In [12]:
df

,l_hand_min_x,l_hand_min_y,l_hand_width,l_hand_height,r_hand_min_x,r_hand_min_y,r_hand_width,r_hand_height,object_min_x,object_min_y,...,r19x,r19y,r20x,r20y,r21x,r21y,lh_depth_dist,rh_depth_dist,picture_name,grasp
0,1920,0,0,0,1001,682,142,64,869,576,...,100,-3,86,-14,78,-27,-128.411926,-2014.397278,salad16085.png,none
1,1920,1080,0,0,1539,608,209,259,667,243,...,504,-358,479,-333,451,-321,-10972.939575,-6049.359009,salad09942.png,none
2,1171,587,118,168,1920,1080,0,0,712,351,...,-334,422,-334,422,-334,422,-3838.286377,-18578.716064,salad11706.png,none
3,1326,521,101,62,0,1080,0,0,1046,267,...,35,403,35,403,35,403,-814.518860,-905.653381,salad04018.png,none
4,1118,351,182,198,1366,647,105,195,564,484,...,296,-71,273,-59,251,-55,-1761.761719,-1246.901855,salad16039.png,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4753,972,270,148,176,0,0,0,0,920,926,...,1023,985,1023,985,1023,985,877.946289,6519.351683,simulation02823.png,precision
4754,877,749,188,121,1003,521,135,97,981,603,...,169,116,166,104,166,94,-2146.814453,-174.041504,simulation00768.png,precision
4755,866,400,172,127,0,0,0,0,943,664,...,1004,727,1004,727,1004,727,-1189.540039,4339.987671,simulation02895.png,precision
4756,1174,875,123,53,0,0,0,0,1097,964,...,1222,1022,1222,1022,1222,1022,901.244385,6482.765343,simulation00537.png,precision


In [13]:
df.to_csv('data/depths_added.csv',index=False)